<div class="alert alert-block alert-warning">
Planning - Acquisition - <font color='red'><b>Preparation</b></font> - Exploratory Analysis - Modeling - Product Delivery
</div>

# Tidy Data Exercises
    
Do your work for this exercise in a jupyter notebook or python script named ```tidy_data```. Save this work in your ```classification-exercises``` repo. Add, commit, and push your changes.

In [1]:
import pandas as pd
import numpy as np
import os
from pydataset import data

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import env
import QMCBT_acquire as acquire
#import QMCBT_prepare as prepare
import QMCBT_explore as explore

# import splitting functions
from sklearn.model_selection import train_test_split

**CUSTOM EXPLORATION FUNCTIONS
nunique_column_all(df): PRINT NUNIQUE OF ALL COLUMNS
nunique_column_objects(df): PRINT NUNIQUE OF COLUMNS THAT ARE OBJECTS
nunique_column_qty(df): PRINT NUNIQUE OF COLUMNS THAT ARE *NOT* OBJECTS
numeric_range(df): COMPUTE RANGE FOR ALL NUMERIC VARIABLES

**USEFUL EXPLORATORY CODE**
DFNAME.head()
DFNAME.shape
DFNAME.shape[0] #read row count
DFNAME.describe().T
DFNAME.columns.to_list()
DFNAME.COLUMNNAME.value_counts(dropna=False)
DFNAME.dtypes
DFNAME.select_dtypes(include='object').columns
DFNAME.select_dtypes(include='float').columns
pd.crosstab(DFNAME.COLUMN-1, DFNAME.COLUMN-2)


### 1. Attendance Data

Read the data from the ```attendance``` table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
url = env.get_db_url('tidy_data')
query = 'SELECT * FROM attendance'

attend_df = pd.read_sql(query, url)
attend_df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attend_df = attend_df.rename(columns={'Unnamed: 0':'name'})
attend_df

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
attend_df = attend_df.melt(id_vars='name', var_name='date', value_name='attendance')
attend_df.head()

,name,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [5]:
attend_df.attendance = attend_df.attendance.map({'P':1, 'H':.5, 'A':0, 'T':.9})
attend_df.head()

,name,date,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [6]:
attend_df.groupby('name').mean()

,attendance
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


### You should end up with something like this:
```
name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64
```

### 2. Coffee Levels

### a. Read the ```coffee_levels``` table.

In [7]:
url = env.get_db_url('tidy_data')
query = 'SELECT * FROM coffee_levels'

coffee_df = pd.read_sql(query, url)
coffee_df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


### b. Transform the data so that each carafe is in it's own column.

In [8]:
coffee_df.pivot_table(index='hour', columns='coffee_carafe')

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

### c. Is this the best shape for the data?

**ANSWER:**  
NO, It was better the way it was so that you can sort and filter by coffee_amount

### 3. Cake Recipes

### a. Read the ```cake_recipes``` table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [9]:
url = env.get_db_url('tidy_data')
query = 'SELECT * FROM cake_recipes'

cake_df = pd.read_sql(query, url)
cake_df

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


### b. Tidy the data as necessary.

In [10]:
cake_df = cake_df.melt(id_vars='recipe:position', var_name='temp', value_name='score')
cake_df.head()

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [11]:
cake_df[['recipe', 'position']] = cake_df['recipe:position'].str.split(':', expand=True)
cake_df.head()

,recipe:position,temp,score,recipe,position
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom


In [12]:
cake_df = cake_df.drop(columns='recipe:position')
cake_df.head()

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom


### c. Which recipe, on average, is the best? 
* recipe b

In [13]:
# WRONG; this is only MAX not max on avg
cake_df[cake_df.score == cake_df.score.max()]

,temp,score,recipe,position
26,300,99.248541,b,bottom


In [14]:
cake_df.groupby('recipe').mean().idxmax()

score    b
dtype: object

### d. Which oven temperature, on average, produces the best results? 
* 275

In [15]:
cake_df.groupby('temp').mean().idxmax()

score    275
dtype: object

In [16]:
cake_df.groupby('temp').mean().sort_values('score').tail(1)

,score
temp,
275,74.886754


### e. Which combination of recipe, rack position, and temperature gives the best result? 
* recipe b, bottom rack, 300 degrees

In [17]:
cake_df.groupby(['recipe', 'position', 'temp']).mean().sort_values('score').tail(1)

,,,score
recipe,position,temp,
b,bottom,300,99.248541


## 4. **Bonus:** 
### Explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.